In [157]:
import pandas as pd

# Replace the file path with your actual file path
file_path = r"C:\Users\njrav\Downloads\UNO Service Learning Data Sheet De-Identified Version.xlsx"

# Read the Excel file
data = pd.read_excel(file_path)

# Display the first few rows of the data
print(data.head())


   Patient ID# Grant Req Date  App Year  ...  Patient Letter Notified? (Directly/Indirectly through rep) Application Signed? Notes
0       180001     2018-10-17         1  ...                                            Missing                      Missing   NaN
1       190001     2019-01-03         1  ...                                            Missing                      Missing   NaN
2       190001     2019-03-11         1  ...                                            Missing                      Missing   NaN
3       190002     2019-05-20         1  ...                                            Missing                      Missing   NaN
4       190003     2019-05-22         1  ...                                            Missing                      Missing   NaN

[5 rows x 30 columns]


In [158]:
import re

# Replace variations of 'ck' with 'Check'
data['Payment Method'] = data['Payment Method'].str.replace(r'(?i)ck[^a-zA-Z]*\d{0,4}', 'Check', regex=True)

data["Payment Submitted?"] = data["Payment Submitted?"].apply(
    lambda x: x.strip().capitalize() if isinstance(x, str) else x
)

# Extract only rows that contain valid dates
data["Payment Date"] = pd.to_datetime(data["Payment Submitted?"], format="%m/%d/%Y", errors="coerce")

# Convert "Payment Submitted?" to "Yes" if it contained a date, otherwise keep it as "No"
data["Payment Submitted?"] = data["Payment Submitted?"].apply(lambda x: "Yes" if pd.to_datetime(x, format="%m/%d/%Y", errors="coerce") not in [None, pd.NaT] else "No")

def clean_city_name(city):
    if pd.isna(city):  # Handle missing values
        return city

    city = city.strip()  # Remove leading/trailing spaces
    city = re.sub(r'[?,()]', '', city)  # Remove unwanted characters
    
    # Standardize "St. Paul"
    if city.lower() in ["saint paul", "st paul"]:
        return "St. Paul"
    
    return city.title()  # Capitalize first letter of each word

# Apply the function to the column
data['Pt City'] = data['Pt City'].apply(clean_city_name)

# Filling NA City Values with "Missing"
data["Pt City"] = data["Pt City"].fillna("Missing")

# Replace specific states with their abbreviations
data["Pt State"] = data["Pt State"].replace({"Florida": "FL", "Iowa": "IA"})

# Capitalize state abbreviations
data["Pt State"] = data["Pt State"].str.upper()

# Fill missing values with "Missing"
data["Pt State"] = data["Pt State"].fillna("MISSING")

# Strip unnecessary spaces
data["Pt State"] = data["Pt State"].str.strip()

# Convert ZIP codes to strings to avoid TypeError
data["Pt Zip"] = data["Pt Zip"].astype(str)

# Keep "Missing" values unchanged while trimming others to 5 digits
data["Pt Zip"] = data["Pt Zip"].apply(lambda x: x if x == "Missing" else str(x)[:5])

# Replace any blanks or 'nan' values with "Missing"
data["Pt Zip"] = data["Pt Zip"].replace(["nan", ""], "Missing").fillna("Missing")

# Capitalize only the first letter of each word
data["Language"] = data["Language"].str.title()

# Fill missing values with "Missing"
data["Language"] = data["Language"].fillna("Missing")

# Strip unnecessary spaces
data["Language"] = data["Language"].str.strip()

# Fix date of birth error
data["DOB"] = data["DOB"].replace("1/9/2973", "1/9/1973")

# Convert all values to strings
data["DOB"] = data["DOB"].astype(str)

# Standardize "Missing" entries
data["DOB"] = data["DOB"].replace(["MISSING", "nan"], "Missing")

# Convert valid dates to MM/DD/YYYY format while keeping "Missing" unchanged
data["DOB"] = data["DOB"].apply(lambda x: pd.to_datetime(x, errors="coerce"))
data["DOB"] = data["DOB"].apply(lambda x: x.strftime("%m/%d/%Y") if pd.notna(x) else "Missing")

# Correct spelling mistake
data["Marital Status"] = data["Marital Status"].replace("Seperated", "Separated")

# Capitalize only the first letter of each word
data["Marital Status"] = data["Marital Status"].str.title()

# Fill missing values with "Missing"
data["Marital Status"] = data["Marital Status"].fillna("Missing")

# Strip unnecessary spaces
data["Marital Status"] = data["Marital Status"].str.strip()

# Fill missing values with "Missing"
data["Gender"] = data["Gender"].fillna("Missing")

# Capitalize only the first letter of each word
data["Gender"] = data["Gender"].str.title()

# Strip unnecessary spaces
data["Gender"] = data["Gender"].str.strip()

# Capitalize only the first letter of each word
data["Race"] = data["Race"].str.title()

# Fill missing values with "Missing"
data["Race"] = data["Race"].fillna("Missing")

# Correct misspellings
data["Race"] = data["Race"].replace({
    "American Indian Or Alaskan Native": "American Indian Or Alaska Native",
    "American Indian Or Alaksa Native": "American Indian Or Alaska Native",
    "Whiate": "White"
})

# Strip unnecessary spaces
data["Hispanic/Latino"] = data["Hispanic/Latino"].str.strip()

# Capitalize first letter of each word
data["Hispanic/Latino"] = data["Hispanic/Latino"].str.title()

# Fill missing values with "Missing"
data["Hispanic/Latino"] = data["Hispanic/Latino"].fillna("Missing")

# Correct misspellings and standardize values
data["Hispanic/Latino"] = data["Hispanic/Latino"].replace({
    "Hispanic Of Latino": "Hispanic Or Latino",
    "Non-Hispanic Latino": "Non-Hispanic Or Latino",
    "Non-Hispanic": "Non-Hispanic Or Latino",
    "No": "Non-Hispanic Or Latino",
    "Yes": "Hispanic Or Latino"
})

# Capitalize first letter of each word
data["Sexual Orientation"] = data["Sexual Orientation"].str.title()

# Correct misspellings and standardize values
data["Sexual Orientation"] = data["Sexual Orientation"].replace({
    "Decline": "Decline To Answer",
    "Female": "Missing",
    "Male": "Missing",
    "Heterosexual": "Straight",
    "Staight": "Straight",
    "Striaght": "Straight",
    "Stright": "Straight",
    "N/A": "Missing"
})

# Fill missing values with "Missing"
data["Sexual Orientation"] = data["Sexual Orientation"].fillna("Missing")

# Strip unnecessary spaces
data["Sexual Orientation"] = data["Sexual Orientation"].str.strip()

# Fill missing values with "Missing"
data["Insurance Type"] = data["Insurance Type"].fillna("Missing")

# Strip unnecessary spaces
data["Insurance Type"] = data["Insurance Type"].str.strip()

# Capitalize first letter of each word
data["Insurance Type"] = data["Insurance Type"].str.title()

# Correct misspellings and standardize values
data["Insurance Type"] = data["Insurance Type"].replace({
    "Medicaid & Medicare": "Medicare & Medicaid",
    "Uninsurred": "Uninsured",
    "Unisured": "Uninsured",
    "Unknown": "Missing"
})

# Strip spaces
data["Household Size"] = data["Household Size"].astype(str).str.strip()

# Standardize missing values
data["Household Size"] = data["Household Size"].replace(["nan", "Missing", "missing"], "Missing").fillna("Missing")

# Convert valid numbers to integers while keeping "Missing" unchanged
data["Household Size"] = data["Household Size"].apply(lambda x: int(x) if x.isnumeric() else x)

# Rid of outlier
data["Household Size"] = data["Household Size"].replace(4602, "Missing")

# Convert numeric values, keeping "Missing" as a string
data["Household Size"] = pd.to_numeric(data["Household Size"], errors="coerce").astype("Int64").astype(str).replace("nan", "Missing")

# Strip spaces
data["Total Household Gross Monthly Income"] = data["Total Household Gross Monthly Income"].astype(str).str.strip()

# Fill blanks with "Missing"
data["Total Household Gross Monthly Income"] = data["Total Household Gross Monthly Income"].replace(["nan", ""], "Missing").fillna("Missing")

# Format numeric values as monetary with two decimals while keeping "Missing" unchanged
data["Total Household Gross Monthly Income"] = data["Total Household Gross Monthly Income"].apply(lambda x: f"{float(x):,.2f}" if x.replace(".", "").isdigit() else x)

# Remove commas before converting to numeric
data["Total Household Gross Monthly Income"] = data["Total Household Gross Monthly Income"].str.replace(",", "", regex=True)

# Convert to numeric
data["Total Household Gross Monthly Income"] = pd.to_numeric(data["Total Household Gross Monthly Income"], errors="coerce")

# Define bins and labels
bins = [0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, float("inf")]
labels = ["$0-$1,000", "$1,001-$2,000", "$2,001-$3,000", "$3,001-$4,000", "$4,001-$5,000", 
          "$5,001-$6,000", "$6,001-$7,000", "$7,001-$8,000", "$8,001-$9,000", "$9,001-$10,000", 
          "$10,001+"]

# Apply binning
data["Monthly Household Income Range"] = pd.cut(data["Total Household Gross Monthly Income"], bins=bins, labels=labels, right=True)


# Strip spaces
data["Distance roundtrip/Tx"] = data["Distance roundtrip/Tx"].astype(str).str.strip()

# Convert non-numeric values and blanks to "Missing"
data["Distance roundtrip/Tx"] = data["Distance roundtrip/Tx"].apply(lambda x: x if x.replace(".", "").isdigit() else "Missing")

# Capitalize first letter of each word
data["Referral Source"] = data["Referral Source"].str.title()

# Remove trailing spaces
data["Referral Source"] = data["Referral Source"].str.strip()

# Fill missing values with "Missing"
data["Referral Source"] = data["Referral Source"].fillna("Missing")

# Fill missing values with "Missing"
data["Referred By:"] = data["Referred By:"].fillna("Missing")

# Convert "Amount" to numeric, coercing errors to NaN
data["Amount"] = pd.to_numeric(data["Amount"], errors="coerce")

# Fill NaN values with 0 (if missing values exist)
data.loc[:, "Amount"] = data["Amount"].fillna(0)

# Convert values that look like dates into "Yes"
data["Patient Letter Notified? (Directly/Indirectly through rep)"] = data["Patient Letter Notified? (Directly/Indirectly through rep)"].apply(
    lambda x: "Yes" if pd.to_datetime(x, errors="coerce") not in [None, pd.NaT] else x
)

# Capitalize first letter of each word
data["Patient Letter Notified? (Directly/Indirectly through rep)"] = data["Patient Letter Notified? (Directly/Indirectly through rep)"].str.title()

# Replace specific missing indicators with "Missing"
data["Patient Letter Notified? (Directly/Indirectly through rep)"] = data["Patient Letter Notified? (Directly/Indirectly through rep)"].replace(["", " ","na", "Na", "Hold", "nan", "HOLD"], "Missing").fillna("Missing")

# Remove trailing spaces
data["Patient Letter Notified? (Directly/Indirectly through rep)"] = data["Patient Letter Notified? (Directly/Indirectly through rep)"].str.strip()

# Strip spaces
data["Application Signed?"] = data["Application Signed?"].astype(str).str.strip()

# Capitalize first letter of each word
data["Application Signed?"] = data["Application Signed?"].str.title()

# Replace "N/A" and blanks with "Missing"
data["Application Signed?"] = data["Application Signed?"].replace(["", "N/A", "Nan"], "Missing").fillna("Missing")

data["DOB"] = pd.to_datetime(data["DOB"], format="%m/%d/%Y", errors="coerce")

data["Age"] = today.year - data["DOB"].dt.year - ( 
    (today.month < data["DOB"].dt.month) | 
    ((today.month == data["DOB"].dt.month) & (today.day < data["DOB"].dt.day))
).astype(int)

# Filter out individuals younger than 18
data = data[data["Age"] >= 18]

# Ensure both columns are datetime
data["Grant Req Date"] = pd.to_datetime(data["Grant Req Date"], errors="coerce")
data["Payment Date"] = pd.to_datetime(data["Payment Date"], errors="coerce")

# Recalculate processing time
data["Processing Time (Days)"] = (data["Payment Date"] - data["Grant Req Date"]).dt.days

# Strip spaces
data["Type of Assistance (CLASS)"] = data["Type of Assistance (CLASS)"].astype(str).str.strip()

# Capitalize first letter of each word
data["Type of Assistance (CLASS)"] = data["Type of Assistance (CLASS)"].str.title()

In [159]:
# Save cleaned dataset to a new Excel file
cleaned_file_path = r"C:\Users\njrav\Downloads\cleaned_data.xlsx"  # Adjust path as needed
data.to_excel(cleaned_file_path, index=False, engine="openpyxl")